In [3]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [4]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [5]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM LEAGUE;", db)

In [6]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


Task_1

In [23]:
players_180_190 = player_data[(player_data['height'] >= 180) & (player_data['height'] <= 190)].shape[0]
players_180_190

5925

In [24]:
assert(isinstance(players_180_190, int))

In [25]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = player_data[player_data['birthday'].dt.year == 1980].shape[0]
players_1980

434

In [26]:
assert(isinstance(players_1980, int))

Task_3

In [27]:
highest_players = player_data.sort_values(by = ['weight', 'player_name'], ascending=[False, True])[:10]['player_name'].tolist()
highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

In [28]:
assert(isinstance(highest_players, list))

Task_4

In [29]:
player_data['birthday_year'] = player_data['birthday'].dt.year
born_years = player_data.groupby('birthday_year').size().loc[lambda x: (1980 <= x.index) & (x.index <= 1990)]
year_born_players = list(born_years.items())
year_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [30]:
assert(len(year_born_players) == 11)
assert(isinstance(year_born_players, list))
for i in range(10):
    assert(isinstance(year_born_players[i], tuple))
    assert(isinstance(year_born_players[i][0], int))
    assert(isinstance(year_born_players[i][0], int))

Task_5

In [33]:
mask = player_data['player_name'].str.startswith('Adriano')
adriano_mean, adriano_std = player_data[mask]['height'].mean(), player_data[mask]['height'].std()
(adriano_mean, adriano_std)

(182.118, 5.361479480723788)

In [34]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

Task_6

In [37]:
dow_with_min_players_born = player_data.groupby(player_data['birthday'].dt.day_name()).size().sort_values().keys()[0]
dow_with_min_players_born

'Sunday'

In [38]:
assert(isinstance(dow_with_min_players_born, str))

Task_7

In [51]:
league = league_data[['id', 'name']]
league.columns = ['league_id', 'league_name']

league_most_matches = match_data \
.merge(league, 'left') \
.groupby('league_name').size() \
.reset_index().sort_values(by=[0, 'league_name'], ascending=[False, True]).iloc[0]['league_name']

league_most_matches

'England Premier League'

In [52]:
assert(isinstance(league_most_matches, str))

Task_8

In [53]:
import re

home_player_cols = [c for c in match_data.columns if re.match(r'home_player_\d\d?', c)]
away_player_cols = [c for c in match_data.columns if re.match(r'away_player_\d\d?', c)]
player_cols = home_player_cols + away_player_cols

max_matches_player_id = match_data \
.melt(value_vars=player_cols) \
.groupby('value').size() \
.sort_values(ascending=False) \
.leys()[0]

max_matches_player = player_data[player_data['player_api_id'] == max_matches_player_id]